In [ ]:
#| default_exp htmx


In [ ]:
#| export
from __future__ import annotations


# HTMX

> Python wrapper of HTMX API.

Currently only implemented `htmx.swap()` manually. If useful, I'll automate wrappers generation from htmx docs.


# Prologue

In [ ]:
#| export
from typing import Literal
from typing import TypeAlias

from olio.common import update_


In [ ]:

import anywidget
import fastcore.all as FC
from fastcore.xml import to_xml


In [ ]:
import inspect

from fastcore.test import *
from rich.console import Console

In [ ]:
from bridget.helpers import cleanupwidgets
from bridget.helpers import Singleling


----


In [ ]:
cprint = (console := Console(width=120)).print


In [ ]:
# needed for vfile:
%load_ext anywidget


In [ ]:
%env ANYWIDGET_HMR=0

env: ANYWIDGET_HMR=0


In [ ]:
from fasthtml.xtend import Script
from IPython.display import HTML

display(HTML(to_xml(Script(src='https://unpkg.com/htmx.org@next/dist/htmx.js'))))


----

# HTMX commander

In [ ]:
class HTMXCommander(Singleling, anywidget.AnyWidget):
    _esm = """
    function initialize({ model }) {
    // debugger;
        model.on("msg:custom", msg => {
            // debugger;
            console.log(`new message: ${JSON.stringify(msg)}`);
            const { cmd, args } = msg;
            if (cmd in htmx) {
                try {
                    htmx[cmd](...(Array.isArray(args) ? args : Object.values(args)));
                } catch (e) {
                    console.error(e);
                }
            } else {
                console.warn(`Unknown HTMX command: ${cmd}`);
            }
        });
        console.log('initialized');
    }
	export default { "initialize": initialize };
    """
    
    def swap(self: HTMXCommander,
        target, 
        content, 
        *, 
        # ---- swapSpec:SwapSpec, 
        swapStyle: Literal['innerHTML','outerHTML','testContent','beforebegin','afterbegin','beforeend','afterend','delete','none'],
        swapDelay: int|None=None, settleDelay: int|None=None,
        transition: bool|None=None,
        # ignoreTitle: bool|None=None, head: Literal['merge', 'append']|None=None,
        scroll: str|None=None, scrollTarget: str|None=None,
        show: str|None=None, showTarget: str|None=None, focusScroll: bool|None=None,
        # ---- swapOptions=None,
        select: str|None=None, selectOOB: str|None=None,
        # eventInfo: dict|None=None,
        anchor: str|None=None,        
        # contextElement: str|None=None,
        # afterSwapCallback: Callable|None=None, afterSettleCallback: Callable|None=None,
    ): ...

    def setup(self, *args, **kwargs):
        super().setup()
        super().__init__(*args, **kwargs)
        dh = display(self, display_id=True)
        if dh: dh.update(HTML('<h3>HTMX commander loaded!</h3>'))  # user feedback, headless widget
        
    def __init__(self, *args, **kwargs):
        self.setup(*args, **kwargs)


In [ ]:
cleanupwidgets('cmdr')

cmdr = HTMXCommander()


## API

### Method - `htmx.swap()` {#swap}

Performs swapping (and settling) of HTML content

##### Parameters

* `target` - the HTML element or string selector of swap target
* `content` - string representation of content to be swapped
* `swapSpec` - swapping specification, representing parameters from `hx-swap`
  * `swapStyle` (required) - swapping style (`innerHTML`, `outerHTML`, `beforebegin` etc)
  * `swapDelay`, `settleDelay` (number) - delays before swapping and settling respectively
  * `transition` (bool) - whether to use HTML transitions for swap
  * `ignoreTitle` (bool) - disables page title updates
  * `head` (string) - specifies `head` tag handling strategy (`merge` or `append`). Leave empty to disable head handling
  * `scroll`, `scrollTarget`, `show`, `showTarget`, `focusScroll` - specifies scroll handling after swap
* `swapOptions` - additional *optional* parameters for swapping
  * `select` - selector for the content to be swapped (equivalent of `hx-select`)
  * `selectOOB` - selector for the content to be swapped out-of-band (equivalent of `hx-select-oob`)
  * `eventInfo` - an object to be attached to `htmx:afterSwap` and `htmx:afterSettle` elements
  * `anchor` - an anchor element that triggered scroll, will be scrolled into view on settle. Provides simple alternative to full scroll handling
  * `contextElement` - DOM element that serves as context to swapping operation. Currently used to find extensions enabled for specific element
  * `afterSwapCallback`, `afterSettleCallback` - callback functions called after swap and settle respectively. Take no arguments


##### Example

```js
    // swap #output element inner HTML with div element with "Swapped!" text
    htmx.swap("#output", "<div>Swapped!</div>", {swapStyle: 'innerHTML'});
```


In [ ]:
#| export

SwapStyleT: TypeAlias = Literal['innerHTML','outerHTML','testContent','beforebegin','afterbegin',
                        'beforeend','afterend','delete','none']


In [ ]:
#| export

def swap(self, 
        target, 
        content, 
        *, 
        # ---- swapSpec:SwapSpec, 
        swapStyle: SwapStyleT='innerHTML',
        swapDelay: int|None=None, settleDelay: int|None=None,
        transition: bool|None=None,
        # ignoreTitle: bool|None=None, head: Literal['merge', 'append']|None=None,
        scroll: str|None=None, scrollTarget: str|None=None,
        show: str|None=None, showTarget: str|None=None, focusScroll: bool|None=None,
        # ---- swapOptions=None,
        select: str|None=None, selectOOB: str|None=None,
        # eventInfo: dict|None=None,
        anchor: str|None=None,        
        # contextElement: str|None=None,
        # afterSwapCallback: Callable|None=None, afterSettleCallback: Callable|None=None,
    ):
    d = {
        'target': target,
        'content': content,
        'swapSpec': update_(**{
            'swapStyle': swapStyle, 'swapDelay': swapDelay, 'settleDelay': settleDelay,
            'transition': transition,
            # 'ignoreTitle': ignoreTitle, 'head': head,
            'scroll': scroll, 'scrollTarget': scrollTarget,
            'show': show, 'showTarget': showTarget, 'focusScroll': focusScroll,
            # 'afterSwapCallback': afterSwapCallback, 'afterSettleCallback': afterSettleCallback,
        }),
        'swapOptions': update_(**{
            'select': select, 'selectOOB': selectOOB,
            # 'eventInfo': eventInfo,
            'anchor': anchor,
            # 'contextElement': contextElement,
            # 'afterSwapCallback': afterSwapCallback, 'afterSettleCallback': afterSettleCallback,
        }),
    }
    self.send({
        'cmd': 'swap',
        'args': [*d.values()]
    })


In [ ]:
FC.patch_to(HTMXCommander)(swap)


## Test swap

In [ ]:
%%HTML

<div id="output-99">Original</div>


In [ ]:
cmdr.swap('#output-99', '<div>Swapped!</div>', swapStyle='innerHTML')


# ----
First steps exploring automation. Skip it.

In [ ]:
sig = inspect.signature(cmdr.swap)
print(sig)
print(sig.parameters)
target_param = sig.parameters['swapStyle']
target_param.name, target_param.default, target_param.annotation, target_param.kind


(target, content, *, swapStyle: 'SwapStyleT' = 'innerHTML', swapDelay: 'int | None' = None, settleDelay: 'int | None' = None, transition: 'bool | None' = None, scroll: 'str | None' = None, scrollTarget: 'str | None' = None, show: 'str | None' = None, showTarget: 'str | None' = None, focusScroll: 'bool | None' = None, select: 'str | None' = None, selectOOB: 'str | None' = None, anchor: 'str | None' = None)
OrderedDict({'target': <Parameter "target">, 'content': <Parameter "content">, 'swapStyle': <Parameter "swapStyle: 'SwapStyleT' = 'innerHTML'">, 'swapDelay': <Parameter "swapDelay: 'int | None' = None">, 'settleDelay': <Parameter "settleDelay: 'int | None' = None">, 'transition': <Parameter "transition: 'bool | None' = None">, 'scroll': <Parameter "scroll: 'str | None' = None">, 'scrollTarget': <Parameter "scrollTarget: 'str | None' = None">, 'show': <Parameter "show: 'str | None' = None">, 'showTarget': <Parameter "showTarget: 'str | None' = None">, 'focusScroll': <Parameter "focusSc

('swapStyle', 'innerHTML', 'SwapStyleT', <_ParameterKind.KEYWORD_ONLY: 3>)

In [ ]:

bb = sig.bind(target='#target', content='<content />', swapStyle='innerHTML', swapDelay=1000)
bb.arguments, bb.signature, bb.args, bb.kwargs


({'target': '#target',
  'content': '<content />',
  'swapStyle': 'innerHTML',
  'swapDelay': 1000},
 <Signature (target, content, *, swapStyle: 'SwapStyleT' = 'innerHTML', swapDelay: 'int | None' = None, settleDelay: 'int | None' = None, transition: 'bool | None' = None, scroll: 'str | None' = None, scrollTarget: 'str | None' = None, show: 'str | None' = None, showTarget: 'str | None' = None, focusScroll: 'bool | None' = None, select: 'str | None' = None, selectOOB: 'str | None' = None, anchor: 'str | None' = None)>,
 ('#target', '<content />'),
 {'swapStyle': 'innerHTML', 'swapDelay': 1000})

In [ ]:
swap_args = {
    'target': 'target',
    'content': 'content',
    '*': '*',
    'swapSpec': {
        'swapStyle': 'swapStyle', 'swapDelay': 'swapDelay', 'settleDelay': 'settleDelay',
        'transition': 'transition',
        # 'ignoreTitle': 'ignoreTitle', 'head': 'head',
        'scroll': 'scroll', 'scrollTarget': 'scrollTarget',
        'show': 'show', 'showTarget': 'showTarget', 'focusScroll': 'focusScroll',
        # 'afterSwapCallback': 'afterSwapCallback', 'afterSettleCallback': 'afterSettleCallback',
    },
    'swapOptions': {
        'select': 'select', 'selectOOB': 'selectOOB',
        # 'eventInfo': 'eventInfo',
        'anchor': 'anchor',
        # 'contextElement': 'contextElement',
        # 'afterSwapCallback': 'afterSwapCallback', 'afterSettleCallback': 'afterSettleCallback',
    },
}


In [ ]:
from inspect import Parameter


swap_args = {
    'target': Parameter('target', Parameter.POSITIONAL_OR_KEYWORD),
    'content': Parameter('content', Parameter.POSITIONAL_OR_KEYWORD),
    'swapSpec': {
        'swapStyle': Parameter('swapStyle', Parameter.KEYWORD_ONLY, annotation=SwapStyleT), 
        'swapDelay': Parameter('swapDelay', Parameter.KEYWORD_ONLY), 
        'settleDelay': Parameter('settleDelay', Parameter.KEYWORD_ONLY),
        'transition': Parameter('transition', Parameter.KEYWORD_ONLY),
        # 'ignoreTitle': 'ignoreTitle', 'head': 'head',
        'scroll': Parameter('scroll', Parameter.KEYWORD_ONLY), 
        'scrollTarget': Parameter('scrollTarget', Parameter.KEYWORD_ONLY),
        'show': Parameter('show', Parameter.KEYWORD_ONLY), 
        'showTarget': Parameter('showTarget', Parameter.KEYWORD_ONLY), 
        'focusScroll': Parameter('focusScroll', Parameter.KEYWORD_ONLY),
        # 'afterSwapCallback': 'afterSwapCallback', 'afterSettleCallback': 'afterSettleCallback',
    },
    'swapOptions': {
        'select': 'select', 'selectOOB': 'selectOOB',
        # 'eventInfo': 'eventInfo',
        'anchor': 'anchor',
        # 'contextElement': 'contextElement',
        # 'afterSwapCallback': 'afterSwapCallback', 'afterSettleCallback': 'afterSettleCallback',
    },
}


In [ ]:
def build_args_dict(args_desc: dict, *args, **kwargs) -> dict:
    "Builds a dictionary from args/kwargs based on an arguments descriptor"
    result = {}
    
    # Process descriptor items in order
    for k, v in args_desc.items():
        if isinstance(v, dict):
            # Handle nested dictionary descriptors
            nested = {
                v2: kwargs.get(k2) 
                for k2, v2 in v.items() 
                if kwargs.get(k2) is not None
            }
            if nested: result[k] = nested
        else:
            # For non-dict values, first try kwargs, then fallback to positional args
            if k in kwargs:
                result[v] = kwargs[k]
            elif len(args) > 0:
                result[v] = args[0]
                args = args[1:]
    
    return result


build_args_dict(swap_args, '#target', '<content />', swapStyle='innerHTML', swapDelay=1000)



{<Parameter "target">: '#target',
 <Parameter "content">: '<content />',
 'swapSpec': {<Parameter "swapStyle: Literal['innerHTML', 'outerHTML', 'testContent', 'beforebegin', 'afterbegin', 'beforeend', 'afterend', 'delete', 'none']">: 'innerHTML',
  <Parameter "swapDelay">: 1000}}

In [ ]:
from IPython.core.getipython import get_ipython


In [ ]:
# Get the current notebook
ip = get_ipython()
if ip:
    notebook = ip.kernel.shell.user_ns['In']  # Access input cells
    notebook_out = ip.kernel.shell.user_ns['Out']  # Access output cells

    # Access specific cells
    input_cell_3 = notebook[3]  # Get content of input cell 3
    output_cell_3 = notebook_out.get(3)  # Get content of output cell 3 (if it exists)

    # Get all cells
    all_input_cells = list(notebook.values())
    all_output_cells = list(notebook_out.values())

    # Get cell numbers
    cell_numbers = list(notebook.keys())

In [ ]:
import nbformat
import os

def get_notebook_path():
    """Get the path of the current notebook"""
    return os.path.abspath(ip.kernel.shell.user_ns['__vsc_ipynb_file__'])  # For VSCode
    # or for classic Jupyter:
    # return os.path.abspath(ip.kernel.shell.user_ns['__notebook_path__'])

nb_path = get_notebook_path()
with open(nb_path) as f:
    nb = nbformat.read(f, as_version=4)
    
# Now you can access all cells including metadata
cells = nb.cells  # List of all cells with full metadata

In [ ]:
cells[26]

{'cell_type': 'code',
 'execution_count': 22,
 'metadata': {},
 'outputs': [{'data': {'text/html': '\n<div id="output-99">Original</div>\n',
    'text/plain': '<IPython.core.display.HTML object>'},
   'metadata': {},
   'output_type': 'display_data'}],
 'source': '%%HTML\n\n<div id="output-99">Original</div>\n'}

# Colophon
----


In [ ]:
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean


In [ ]:
if FC.IN_NOTEBOOK:
    nb_path = '21_htmx.ipynb'
    nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)
